# Iterables and Iterators
### _or_




# Round and Round the Mulberry Bush
-


### Steve Holden

In [ ]:
def do_something_with(o):
    "Acts as a proxy for real work of any kind."
    print("---", o, "---")

test_list = ["Roberta", "Tom", "Alice"]
do_something_with(test_list)

In [ ]:
for item in test_list:
    do_something_with(item)

## Iteration History

### Back before you could iterate over dictionaries ... (v1.5.2?)

In [ ]:
# How "for i in test_list" used to work (and still can)
_private_var = 0
while True:
    try:
        i = test_list.__getitem__(_private_var)
    except IndexError:
        break
    do_something_with(i)
    _private_var += 1       

### Writing an _Old-Style_ Iterable

In [ ]:
class Stars():
    "Class with only __init__ and __getitem__."
    def __init__(self, N):
        self.N = N
    def __getitem__(self, index):
        print("Getting item:", index)  # trace print
        if index >= self.N:
            raise IndexError
        return "*" * index

s = Stars(3)

for v in s:
    do_something_with(v)

## Enter the _modern-day_ Iterable

In [ ]:
for i in None:
    do_something_with(i)

#### "is not iterable?"

## How does Python “Know” Something is Iterable??

## What can we iterate over, but not subscript?

In [ ]:
def g(): yield 42

set(dir(g())) & set(dir(tuple())) - set(dir(object))

## So What does `__iter__` do?

In [ ]:
tli = test_list.__iter__()
type(tli)

#### It returns an _iterator_ - in this case a list iterator

#### Coincidentally, this is why you can't iterate over `None`

In [ ]:
print("__iter__" in dir(None), "__getitem__" in dir(None))

## How Iteration works today (mostly)

In [ ]:
def iterate_over(something):
    _i = something.__iter__()  # creates an iterator
    while True:
        try:
            i = _i.__next__()
        except StopIteration: # iterator is exhausted
            break
        do_something_with(i)

In [ ]:
iterate_over(test_list)

## If objects have no `__iter__` method ...
###### ... Python still attempts to fall back to `__getitem__`

In [ ]:
hasattr(test_list, "__iter__")

In [ ]:
li = test_list.__iter__()
print(li, type(li))

## Recognizing Iterators and Iterables

In [ ]:
# Iterators have both __iter__ and __next__
print(hasattr(tli, '__iter__'), hasattr(tli, '__next__'))

In [ ]:
# Iterables only have __iter__ (or possibly __getitem__)
print(hasattr(test_list, '__iter__'), hasattr(test_list, '__next__'))

## A Quick Piece of Shorthand

## _`iter(thing)`_
## is the same as
## _`thing.__iter__()`_

## This is the easy way to create an iterator from an iterable!

## Iterating over Iterables _vs_ Iterators

In [ ]:
# Create two distinct iterators
iterator_1 = iter(test_list) # same as test_list.__iter__()
iterator_2 = iter(test_list)
print(id(iterator_1), id(iterator_2), sep="\n")
print(iterator_1 is iterator_2)

### Nested iterations over iterables

In [ ]:
for i in test_list:
    for j in test_list:
        do_something_with(f'{i} : {j}')

### Nested iterations over two separate iterators

In [ ]:
iterator_1 = iter(test_list)
iterator_2 = iter(test_list)
for i in iterator_1:
    print("outer loop")
    for j in iterator_2:
        print("inner loop")
        do_something_with(i + ":" + j)

### Nested iterations over _the same_ iterator

In [ ]:
iterator_1 = iter(test_list)
for i in iterator_1:
    print("outer loop")
    for j in iterator_1:
        print("inner loop")
        do_something_with(i + ":" + j)

### Using Iterators Doesn't Mean No Issues

In [ ]:
it_4 = iter(["one", "two", "three", "four"])
it_5 = iter(["five", "six", "seven"])
for iterator in it_4, it_5:
    print("++ New iterator ++")
    for item_1 in iterator:
        item_2 = next(iterator)
        do_something_with(f'{item_1} : {item_2}')

## Writing Your Own Iterators and Iterables

In [ ]:
def is_iterable(o):
    "Return True if o is an iterable."
    return hasattr(o, "__iter__") and not hasattr(o, "__next__")

def is_iterator(o):
    "Return True if o is an iterator."
    return hasattr(o, "__iter__") and hasattr(o, "__next__")

In [ ]:
test_it = iter(test_list)
print(is_iterable(test_list), is_iterator(test_list))
print(is_iterable(test_it), is_iterator(test_it))

### The Basic Iterator Pattern

In [ ]:
class MyIterator:
    "An iterator to produce each character of a string N times."
    def __init__(self, s, N):
        self.s = s
        self.N =  N
        self.pos = self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.pos >= len(self.s):
            raise StopIteration
        result = self.s[self.pos]
        self.count += 1
        if self.count == self.N:
            self.pos += 1
            self.count = 0
        return result

In [ ]:
for s in MyIterator("abc", 2):
    do_something_with(s)

In [ ]:
it_6 = MyIterator("*+", 3)
it_7 = MyIterator("=-", 3)
for c1 in it_6:
    print("iterating over c1:", c1)
    for c2 in it_7:
        do_something_with(c1+":"+c2)

## The Basic Iterable Pattern

In [ ]:
class MIString(str):
    def __new__(cls, value, N):
        return str.__new__(cls, value)
    def __init__(self, value, N):
        self.N = N
    def __iter__(self):
        return MyIterator(self, self.N)

In [ ]:
[s for s in MIString("xyz", 3)]

## A Short Example

In [ ]:
x = MIString("01", 2)
for c1 in x:
    for c2 in x:
        print(c1, c2)

In [ ]:
is_iterable(x), is_iterator(x), is_iterable(iter(x)), is_iterator(iter(x))

In [ ]:
class MIString2(str):
    def __new__(cls,value, N):
        return str.__new__(cls, value)
    def __init__(self, value, N):
        self.N = N
    def __iter__(self):
        for c in str(self):
            for i in range(self.N):
                yield c

In [ ]:
[c for c in MIString2("abcde", 3)]

### Python Iterables

In [ ]:
is_iterable({}), is_iterable(()), is_iterable(set()), is_iterable("")

## Any others?